In [1]:
%pwd

'/home/ahmed/CV_ADD_PROJECT/MachineLearning/Telco-Customer-Churn-Prediction-AWS-DVC-MLflow-/notebook'

In [2]:
Data_Base_Name = 'Churn'

In [3]:
import pymongo
import os
import sys
import certifi
from dotenv import load_dotenv

load_dotenv()
ca = certifi.where()

class MongoDBClient:

    def __init__(self, database=Data_Base_Name): 
        try:
            mongodb_url = os.getenv('MONGODB_URL')
            if not mongodb_url:
                #logging.info('Missing mongodb_url')
                raise ValueError('MongoDB URL is not found')
            
            self.client = pymongo.MongoClient(mongodb_url, tlsCAFile=ca)
            self.database = self.client[database]
            self.database_name = database

        
        except Exception as e:
            raise(e)
            #raise CustomException(e, sys)


In [4]:
from dotenv import load_dotenv
import pandas as pd
import pymongo
import certifi
import json
import sys
import os

# Load environment variables
load_dotenv()

ca = certifi.where()
MONGODB_URL = os.getenv('MONGODB_URL')

Data_Base_Name = "Churn"  

class Data_Insert_Mongo:

    def __init__(self):
        try:
            self.mongo_client = None
        except Exception as e:
            raise Exception(e)

    def csv_to_json(self, file_path):
        try:
            if not os.path.exists(file_path):
                raise FileNotFoundError("CSV file not found at given path")

            data = pd.read_csv(file_path)
            data.reset_index(drop=True, inplace=True)
            records = json.loads(data.to_json(orient="records"))
            return records

        except Exception as e:
            raise Exception(e)

    def insert_data_into_mongodb(self, records, database, collection):
        try:
            if not records:
                return 0
            if not database or not collection:
                raise ValueError("Database and Collection names cannot be empty")

            self.mongo_client = pymongo.MongoClient(MONGODB_URL, tlsCAFile=ca)
            db = self.mongo_client[database]
            coll = db[collection]
            result = coll.insert_many(records)
            return len(result.inserted_ids)

        except Exception as e:
            raise Exception(e)
        finally:
            if self.mongo_client:
                self.mongo_client.close()

if __name__ == '__main__':
    try:
        data_path = os.path.join(os.getcwd(),'Telco_Customer_Churn.csv')
        database_name = Data_Base_Name
        collection = 'Churn'

        data_obj = Data_Insert_Mongo()
        records = data_obj.csv_to_json(data_path)
        print(len(records))
        record_count = data_obj.insert_data_into_mongodb(records, database_name, collection)
        print(f'Inserted {record_count} records into MongoDB')

    except Exception as e:
        raise Exception(e)


7043
Inserted 7043 records into MongoDB
